In [1]:
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import pandas as pd
import numpy as np
import jinja2
import json

import mpld3
from mpld3 import plugins, utils, fig_to_html
mpld3.enable_notebook()

In [2]:
df_4bedroom = pd.DataFrame.from_csv('../ZillowData/Neighborhood_Zhvi_4bedroom.csv')
df = df_4bedroom.query('Metro=="San Francisco"')

In [3]:
relevant_columns = list(df)
relevant_columns.remove('State') 
relevant_columns.remove('Metro')
relevant_columns.remove('SizeRank')

df = df[relevant_columns]
dates = relevant_columns[3:]

matrix = df.as_matrix()
x = [dt.datetime.strptime(d,'%Y-%m').date() for d in dates]

In [4]:
# print(matrix[:,0])

In [5]:
neigh_city = matrix[:,:2]
labels = ['%s, %s' %(neigh_city[i,0], neigh_city[i,1]) for i in range(len(neigh_city))]

In [6]:
# print (labels)

In [7]:
median_prices = matrix[:,3:]

In [8]:
class HighlightLines(plugins.PluginBase):
    """A plugin to highlight lines on hover"""

    JAVASCRIPT = """
    mpld3.register_plugin("linehighlight", LineHighlightPlugin);
    LineHighlightPlugin.prototype = Object.create(mpld3.Plugin.prototype);
    LineHighlightPlugin.prototype.constructor = LineHighlightPlugin;
    LineHighlightPlugin.prototype.requiredProps = ["line_ids"];
    LineHighlightPlugin.prototype.defaultProps = {alpha_bg:0.3, alpha_fg:1.0}
    function LineHighlightPlugin(fig, props){
        mpld3.Plugin.call(this, fig, props);
    };

    LineHighlightPlugin.prototype.draw = function(){
      for(var i=0; i<this.props.line_ids.length; i++){
         var obj = mpld3.get_element(this.props.line_ids[i], this.fig),
             alpha_fg = this.props.alpha_fg;
             alpha_bg = this.props.alpha_bg;
         obj.elements()
             .on("mouseover", function(d, i){
                            d3.select(this).transition().duration(50)
                              .style("stroke-opacity", alpha_fg); })
             .on("mouseout", function(d, i){
                            d3.select(this).transition().duration(200)
                              .style("stroke-opacity", alpha_bg); });
      }
    };
    """

    def __init__(self, lines):
        self.lines = lines
        self.dict_ = {"type": "linehighlight",
                      "line_ids": [utils.get_id(line) for line in lines],
                      "alpha_bg": lines[0].get_alpha(),
                      "alpha_fg": 1.0}
        
class LineLabelTooltip(plugins.PluginBase):
    """A Plugin to enable a tooltip: text which hovers over a line.

    Parameters
    ----------
    line : matplotlib Line2D object
        The figure element to apply the tooltip to
    label : string
        If supplied, specify the labels for each point in points.  If not
        supplied, the (x, y) values will be used.
    hoffset, voffset : integer
        The number of pixels to offset the tooltip text.  Default is
        hoffset = 0, voffset = 10

    Examples
    --------
    >>> import matplotlib.pyplot as plt
    >>> from mpld3 import fig_to_html, plugins
    >>> fig, ax = plt.subplots()
    >>> lines = ax.plot(range(10), 'o')
    >>> plugins.connect(fig, LineLabelTooltip(lines[0]))
    >>> fig_to_html(fig)
    """
    def __init__(self, points, label=None,
                 hoffset=0, voffset=10, location="mouse"):
        if location not in ["bottom left", "top left", "bottom right",
                            "top right", "mouse"]:
            raise ValueError("invalid location: {0}".format(location))
        self.dict_ = {"type": "tooltip",
                      "id": utils.get_id(points),
                      "labels": label if label is None else [label],
                      "hoffset": hoffset,
                      "voffset": voffset,
                      "location": location}

fig, ax = plt.subplots(subplot_kw={})
lines = ax.plot(x, median_prices.T, color=sns.xkcd_rgb["denim blue"], lw=4, alpha=0.1)
ax.set_xlabel = 'Median Price'
ax.set_ylabel = 'Year'
ax.set_title = '4 Bedroom Homes in the Bay Area'

plugins.connect(fig, HighlightLines(lines))
plugins.connect(fig, LineLabelTooltip(lines[0], labels))

#fig_to_html(fig)
mpld3.show()


Note: if you're in the IPython notebook, mpld3.show() is not the best command
      to use. Consider using mpld3.display(), or mpld3.enable_notebook().
      See more information at http://mpld3.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [09/Dec/2016 16:57:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2016 16:57:40] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2016 16:57:40] "GET /mpld3.js HTTP/1.1" 200 -



stopping Server...
